<a href="https://colab.research.google.com/github/bayarra/cs598-dlh/blob/main/DLH_240_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Paper 240. Application of deep and machine learning techniques for multi-label classification performance on psychotic disorder diseases

##PART 1: Set up packages needed for project, Load data into Pandas Dataframe.

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
#drive.mount('/content/drive/u/2/folders/1yJUnHaHFQVsBSPvL9oIxB9Ta84Xv-U_D')\
#https://drive.google.com/drive/u/2/folders/1yJUnHaHFQVsBSPvL9oIxB9Ta84Xv-U_D
drive.mount('/content/drive/')
#uploaded=files.upload()
#!ls
path = "/content/drive/MyDrive/mmc240_c.csv"
df = pd.read_csv(path)
df

Mounted at /content/drive/


,sex,age,faNoily_status,religion,occupation,genetic,status,loss_of_parent,divorse,Injury,Spiritual_consult,Insominia,shizopherania,vascula_demetia,MBD,Bipolar,agecode
0,M,18,Yes,C,STUDENT,Yes,S,Yes,No,No,Yes,N,P,P,P,N,1
1,F,30,Yes,M,ARTISAN,Yes,S,Yes,No,Yes,Yes,P,P,P,N,N,1
2,M,22,Yes,C,STUDENT,No,S,No,No,No,Yes,P,P,P,N,P,1
3,M,35,No,M,ARTISAN,No,M,No,No,No,Yes,P,P,N,N,P,2
4,M,30,Yes,M,ARTISAN,Yes,M,No,No,No,Yes,P,P,P,P,P,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,F,73,Yes,M,RETIRED,Yes,S,Yes,No,No,Yes,P,N,P,N,P,3
496,F,50,No,M,ARTISAN,No,M,Yes,No,No,No,P,P,N,P,P,2
497,F,32,No,C,FORCE,No,M,No,No,No,Yes,N,P,P,P,N,2
498,M,13,Yes,C,STUDENT,No,S,Yes,No,No,No,N,P,N,N,N,1


## PART 2: Label encode the CSV data.
AKA convert String/char values from the CSV into binary values

First, we will convert the columns that contain basic values (such as M/F, Yes/No, or True/False) into binary values (0/1) and replace them in the pandas dataframe.

In [ ]:
from sklearn.preprocessing import LabelEncoder

"""Create the Label Encoder"""
label_encoder = LabelEncoder()

"""Convert Sex values (M/F) to binary values"""
sex_label = label_encoder.fit_transform(df["sex"])
Data = df.drop("sex", axis='columns')
Data["sex"] = sex_label

"""Convert faNoily_status column (Yes/No) to binary values"""
family_status = label_encoder.fit_transform(Data["faNoily_status"])
Data2 = Data.drop("faNoily_status", axis='columns')
Data2["family_status"] = family_status

"""Convert genetic column (Yes/No) to binary values"""
genetic_status = label_encoder.fit_transform(Data2["genetic"])
Data3 = Data2.drop("genetic", axis='columns')
Data3["genetic"] = genetic_status

"""Convert status column (S/M) to binary values"""
status = label_encoder.fit_transform(Data3["status"])
Data3 = Data3.drop("status", axis='columns')
Data3["status"] = status

"""Loss of Parent"""
loss_of_parent = label_encoder.fit_transform(Data3["loss_of_parent"])
Data4 = Data3.drop("loss_of_parent", axis='columns')
Data4["loss_of_parent"] = loss_of_parent

"""Divorce"""
divorce = label_encoder.fit_transform(Data4["divorse"])
Data5 = Data4.drop("divorse", axis='columns')
Data5["divorce"] = divorce

"""Injury"""
injury = label_encoder.fit_transform(Data5["Injury"])
Data6 = Data5.drop("Injury", axis='columns')
Data6["Injury"] = injury

"""Spiritual Consult"""
spiritual_consult = label_encoder.fit_transform(Data6["Spiritual_consult"])
Data7 = Data6.drop("Spiritual_consult", axis='columns')
Data7["Spiritual_consult"] = spiritual_consult

#TODO: Still need to to label encoding on the rows with weird values (such as Bipolar, dementia, shizophrenia, etc)


Now that the basic columns have been converted, the more complex rows with multiple values (such as religion, occupation, etc) now need to be encoded one-hot encoding.

In [ ]:
"""religion"""
one_hot = pd.get_dummies(Data7['religion'], prefix='religion')
Data8 = Data7.drop("religion", axis='columns')
Data8 = Data8.join(one_hot)

one_hot = pd.get_dummies(Data8['occupation'], prefix='occupation')
Data9 = Data8.drop("occupation", axis='columns')
Data9 = Data9.join(one_hot)

one_hot = pd.get_dummies(Data9['agecode'], prefix='agecode')
Data9 = Data9.drop("agecode", axis='columns')
Data9 = Data9.join(one_hot)

"""
religion = one_hot_encoder.fit_transform(Data7[["religion"]])
Data8 = Data7.drop("religion", axis='columns')
Data8[["religion"]] = religion

occupation = one_hot_encoder.fit_transform(Data8[["occupation"]])
Data9 = Data8.drop("occupation", axis='columns')
Data9[["occupation"]] = occupation
"""

Data9

,age,Insominia,shizopherania,vascula_demetia,MBD,Bipolar,sex,family_status,genetic,status,...,religion_O,occupation_ARTISAN,occupation_C/SERVANT,occupation_FORCE,occupation_RETIRED,occupation_STUDENT,occupation_UNEMPLYD,agecode_1,agecode_2,agecode_3
0,18,N,P,P,P,N,1,1,1,1,...,0,0,0,0,0,1,0,1,0,0
1,30,P,P,P,N,N,0,1,1,1,...,0,1,0,0,0,0,0,1,0,0
2,22,P,P,P,N,P,1,1,0,1,...,0,0,0,0,0,1,0,1,0,0
3,35,P,P,N,N,P,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,30,P,P,P,P,P,1,1,1,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,73,P,N,P,N,P,0,1,1,1,...,0,0,0,0,1,0,0,0,0,1
496,50,P,P,N,P,P,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
497,32,N,P,P,P,N,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
498,13,N,P,N,N,N,1,1,0,1,...,0,0,0,0,0,1,0,1,0,0


Now that Y columns have been converted, some has multiple values, for example for the vascula demetia, it has 4 values:  'N', 'N       ­>', 'P', 'P       ­>'. We cleaned up those unicode characters and keep the only values N,P

In [ ]:
#Insominia N|P
insominia = label_encoder.fit_transform(Data9["Insominia"])
data = Data9.drop("Insominia", axis='columns')
data["Insominia"] = insominia

#shizopherania N|P
shizopherania = label_encoder.fit_transform(data["shizopherania"])
data = data.drop("shizopherania", axis='columns')
data["shizopherania"] = shizopherania

#vascula demetia N|P
vascula_demetia = label_encoder.fit_transform(data["vascula_demetia"])
data = data.drop("vascula_demetia", axis='columns')
data["vascula_demetia"] = vascula_demetia

#MBD N|P
MBD = label_encoder.fit_transform(Data9["MBD"])
data = data.drop("MBD", axis='columns')
data["MBD"] = MBD

#Bipolar N|P
Bipolar = label_encoder.fit_transform(Data9["Bipolar"])
data = data.drop("Bipolar", axis='columns')
data["Bipolar"] = Bipolar

data_orig = data.copy()
data_feat = data.copy()
del data_feat['Insominia']
del data_feat['shizopherania']
del data_feat['vascula_demetia']
del data_feat['MBD']
del data_feat['Bipolar']

data['target']=data['Insominia'].astype(str)+data['shizopherania'].astype(str)+data['vascula_demetia'].astype(str)+data['MBD'].astype(str)+data['Bipolar'].astype(str)

data = data.groupby('target').filter(lambda x: len(x) > 6)

data3 = data.copy()

data2 = data.copy()

del data['target']


# remove these values since they are now represented by 'target'
del data2['Insominia']
del data2['shizopherania']
del data2['vascula_demetia']
del data2['MBD']
del data2['Bipolar']


data2

,age,sex,family_status,genetic,status,loss_of_parent,divorce,Injury,Spiritual_consult,religion_C,...,occupation_ARTISAN,occupation_C/SERVANT,occupation_FORCE,occupation_RETIRED,occupation_STUDENT,occupation_UNEMPLYD,agecode_1,agecode_2,agecode_3,target
0,18,1,1,1,1,1,0,0,1,1,...,0,0,0,0,1,0,1,0,0,01110
2,22,1,1,0,1,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,11101
3,35,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,11001
4,30,1,1,1,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,11111
5,86,0,1,0,0,1,0,0,1,1,...,0,0,0,1,0,0,0,0,1,01100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,73,0,1,1,1,1,0,0,1,0,...,0,0,0,1,0,0,0,0,1,10101
496,50,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,11011
497,32,0,0,0,0,0,0,0,1,1,...,0,0,1,0,0,0,0,1,0,01110
498,13,1,1,0,1,1,0,0,0,1,...,0,0,0,0,1,0,1,0,0,01000


#PART 3: Create Test/Training Set from Data

In [ ]:
from sklearn.model_selection import train_test_split
#TODO: What labels are we predicting? If multiple, do we need to create multiple sets?
y = data.pop('Insominia')
X = data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train

,age,sex,family_status,genetic,status,loss_of_parent,divorce,Injury,Spiritual_consult,religion_C,...,occupation_RETIRED,occupation_STUDENT,occupation_UNEMPLYD,agecode_1,agecode_2,agecode_3,shizopherania,vascula_demetia,MBD,Bipolar
217,39,1,1,0,1,1,0,0,1,0,...,0,0,0,0,1,0,1,1,0,0
490,55,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,1,0,1,1
476,27,0,1,1,1,0,0,0,1,0,...,0,0,0,1,0,0,1,1,0,0
5,86,0,1,0,0,1,0,0,1,1,...,1,0,0,0,0,1,1,1,0,0
481,27,0,0,0,1,1,0,0,1,0,...,0,0,1,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,27,0,1,0,1,0,0,1,1,0,...,0,1,0,1,0,0,1,1,0,0
121,34,0,0,0,0,0,0,0,1,1,...,0,0,1,0,1,0,1,1,0,0
390,45,0,1,1,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
252,24,1,0,1,1,0,0,0,1,1,...,0,1,0,1,0,0,1,1,1,0


In [ ]:
df['Insominia'].value_counts()/len(df)

N    0.594
P    0.406
Name: Insominia, dtype: float64

##PART 4: Balance Insomnia dataset with SMOTE



In [ ]:
#TODO: How many random samples do we need?
smote = SMOTE(random_state=101)
X_train_new, y_train_new = smote.fit_resample(X_train, y_train.ravel())
X_train_new

,age,sex,family_status,genetic,status,loss_of_parent,divorce,Injury,Spiritual_consult,religion_C,...,occupation_RETIRED,occupation_STUDENT,occupation_UNEMPLYD,agecode_1,agecode_2,agecode_3,shizopherania,vascula_demetia,MBD,Bipolar
0,39,1,1,0,1,1,0,0,1,0,...,0,0,0,0,1,0,1,1,0,0
1,55,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,1,0,1,1
2,27,0,1,1,1,0,0,0,1,0,...,0,0,0,1,0,0,1,1,0,0
3,86,0,1,0,0,1,0,0,1,1,...,1,0,0,0,0,1,1,1,0,0
4,27,0,0,0,1,1,0,0,1,0,...,0,0,1,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,28,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,1,1,1,1
406,67,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
407,42,0,0,1,0,0,1,0,0,1,...,0,0,0,0,1,0,1,0,0,1
408,43,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1


#Part 5: Multi-Label Classification with non-balanced data using Classic ML models

In [ ]:
data_temp = data2.copy()
y_5 = data_temp.pop('target')
X_5 = data_temp
X_train, X_test, y_train, y_test = train_test_split(X_5, y_5, test_size=0.2)


MLP = MLPClassifier(alpha=1, max_iter=1000)
MLP.fit(X_train, y_train)
score_MLP = MLP.score(X_test, y_test)
print("MLP: ", score_MLP)

svc = LinearSVC(C=0.025, max_iter=10000)
svc.fit(X_train, y_train)
score_svm = svc.score(X_test, y_test)
print("SVM: ", score_svm)

#RF = RandomForestClassifier(max_depth=5, n_estimators=100, max_features=1)
RF = RandomForestClassifier(max_depth=5, n_estimators=100)
RF.fit(X_train, y_train)
score_RF = RF.score(X_test, y_test)
print("RandomForest: ", score_RF)

decistion_tree = DecisionTreeClassifier(max_depth=5)
decistion_tree.fit(X_train, y_train)
score_decistion_tree = decistion_tree.score(X_test, y_test)
print("DecitionTree: ", score_decistion_tree)


MLP:  0.44329896907216493
SVM:  0.41237113402061853
RandomForest:  0.44329896907216493
DecitionTree:  0.422680412371134


#Part 6: Multi-Label Classification with balanced data using Classic ML models


In [ ]:
data_temp = data2.copy()
#print(data2)
y_6 = data_temp.pop('target')
X_6 = data_temp
X_train, X_test, y_train, y_test = train_test_split(X_6, y_6, test_size=0.2)

"""TODO: Fix smote here"""
smote = SMOTE(random_state=101)
X_train_new, y_train_new = smote.fit_resample(X_train, y_train.ravel())

MLP = MLPClassifier(alpha=1, max_iter=1000)
MLP.fit(X_train_new, y_train_new)
score_MLP = MLP.score(X_test, y_test)
print(f"MLP score is : ", score_MLP)

svc = LinearSVC(C=0.025, max_iter=10000)
svc.fit(X_train_new, y_train_new)
score_svm = svc.score(X_test, y_test)
print(f"SVM score is: ", score_svm)

#RF = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
RF = RandomForestClassifier(max_depth=5, n_estimators=100)
RF.fit(X_train_new, y_train_new)
score_RF = RF.score(X_test, y_test)
print(f"RandomForest score is: ", score_RF)

decistion_tree = DecisionTreeClassifier(max_depth=5)
decistion_tree.fit(X_train_new, y_train_new)
score_decistion_tree = decistion_tree.score(X_test, y_test)
print("DecitionTree score is: ", score_decistion_tree)

MLP score is :  0.41237113402061853
SVM score is:  0.422680412371134
RandomForest score is:  0.4329896907216495
DecitionTree score is:  0.3917525773195876


#Part 7: Single Label Classificaiton With non-balanced Neural Network



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense,Input,Dropout,Flatten,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import set_image_data_format

target_names = ['Insominia', 'shizopherania', 'vascula_demetia', 'MBD', 'Bipolar']

def NeuralNetwork_A(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  X_train_test, X_validation, y_train_test, y_validation = train_test_split(X_train, y_train, test_size=0.3)



  model = Sequential()
  model.add(Dense(15, input_dim=25, activation='relu', name='Input'))
  model.add(Dropout(0.4))
  model.add(Dense(20, activation='relu', name='Hidden1'))
  model.add(Dropout(0.4))
  model.add(Dense(40, activation='relu', name='Hidden2'))
  model.add(Dropout(0.4))
  model.add(Dense(50, activation='relu', name='Hidden3'))
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid', name='Output'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  X_train_np = np.asarray(X_train_test).astype('int')

  X_train_tensor = tf.convert_to_tensor(X_train_np)
  y_train_tensor = tf.convert_to_tensor(y_train_test)

  #print(X_train_tensor.shape)
  #print(y_train_tensor.shape)

  model.fit(X_train_tensor, y_train_tensor, epochs=40, batch_size=10)


  # evaluate the model with the validation data
  X_validate_np = np.asarray(X_validation).astype('int')

  X_validate_tensor = tf.convert_to_tensor(X_validate_np)
  y_validate_tensor = tf.convert_to_tensor(y_validation)

  r_val = model.evaluate(X_validate_tensor, y_validate_tensor, batch_size=10)

  #evaluate the model with test data
  X_test_np = np.asarray(X_test).astype('int')

  X_test_tensor = tf.convert_to_tensor(X_test_np)
  y_test_tensor = tf.convert_to_tensor(y_test)

  r_test = model.evaluate(X_test_tensor, y_test_tensor, batch_size=10)

  return r_val, r_test

def prep_data(X, target):
  y = X.pop(target)
  return X, y

results1 = []
for t in target_names:
  x_i = data_orig.copy()
  x_i, y_i = prep_data(x_i, t)
  r_val, r_test = NeuralNetwork_A(x_i, y_i)
  results1.append(f"{t} - Validation loss & accuracy: {r_val}")
  results1.append(f"{t} - Test loss & accuracy: {r_test}")
for r in results1:
  print(r)


Epoch 1/40
28/28 [==============================] - 1s 2ms/step - loss: 1.5287 - accuracy: 0.4929
Epoch 2/40
28/28 [==============================] - 0s 2ms/step - loss: 1.1714 - accuracy: 0.5286
Epoch 3/40
28/28 [==============================] - 0s 3ms/step - loss: 1.0433 - accuracy: 0.5143
Epoch 4/40
28/28 [==============================] - 0s 3ms/step - loss: 0.9197 - accuracy: 0.5250
Epoch 5/40
28/28 [==============================] - 0s 3ms/step - loss: 0.9046 - accuracy: 0.5071
Epoch 6/40
28/28 [==============================] - 0s 2ms/step - loss: 0.7842 - accuracy: 0.5143
Epoch 7/40
28/28 [==============================] - 0s 2ms/step - loss: 0.7229 - accuracy: 0.5964
Epoch 8/40
28/28 [==============================] - 0s 2ms/step - loss: 0.7687 - accuracy: 0.5357
Epoch 9/40
28/28 [==============================] - 0s 2ms/step - loss: 0.7681 - accuracy: 0.6071
Epoch 10/40
28/28 [==============================] - 0s 2ms/step - loss: 0.7246 - accuracy: 0.5679
Epoch 11/40
28/28 [

#Part8: Single Label Classification with balanced data using Neural Network

In [ ]:
def NeuralNetwork_B(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  X_train_test, X_validation, y_train_test, y_validation = train_test_split(X_train, y_train, test_size=0.3)

  # SMOTE the training data
  smote = SMOTE(random_state=101)
  X_train_new, y_train_new = smote.fit_resample(X_train_test, y_train_test.ravel())

  # SMOTE the testing data
  X_validate_new, y_validate_new = smote.fit_resample(X_validation, y_validation.ravel())


  model = Sequential()
  model.add(Dense(15, input_dim=25, activation='relu', name='Input'))
  model.add(Dropout(0.4))
  model.add(Dense(20, activation='relu', name='Hidden1'))
  model.add(Dropout(0.4))
  model.add(Dense(40, activation='relu', name='Hidden2'))
  model.add(Dropout(0.4))
  model.add(Dense(50, activation='relu', name='Hidden3'))
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid', name='Output'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # conver the data to the proper tensor type
  X_train_np = np.asarray(X_train_new).astype('int')

  X_train_tensor = tf.convert_to_tensor(X_train_np)
  y_train_tensor = tf.convert_to_tensor(y_train_new)

  #train the model
  model.fit(X_train_tensor, y_train_tensor, epochs=40, batch_size=10)

  # evaluate the model with the validation data
  X_validate_np = np.asarray(X_validate_new).astype('int')

  X_validate_tensor = tf.convert_to_tensor(X_validate_np)
  y_validate_tensor = tf.convert_to_tensor(y_validate_new)

  r_val = model.evaluate(X_validate_tensor, y_validate_tensor, batch_size=10)

  #evaluate the model with test data
  X_test_np = np.asarray(X_test).astype('int')

  X_test_tensor = tf.convert_to_tensor(X_test_np)
  y_test_tensor = tf.convert_to_tensor(y_test)

  r_test = model.evaluate(X_test_tensor, y_test_tensor, batch_size=10)

  return r_val, r_test

results2 = []
for t in target_names:
  x_i = data3.copy()
  del x_i['target']
  x_i, y_i = prep_data(x_i, t)
  r_val, r_test = NeuralNetwork_B(x_i, y_i)
  results2.append(f"{t} - Validation loss & accuracy: {r_val}")
  results2.append(f"{t} - Test loss & accuracy: {r_test}")
for r in results2:
  print(r)


Epoch 1/40
33/33 [==============================] - 1s 2ms/step - loss: 1.6425 - accuracy: 0.4788
Epoch 2/40
33/33 [==============================] - 0s 2ms/step - loss: 1.0666 - accuracy: 0.5000
Epoch 3/40
33/33 [==============================] - 0s 2ms/step - loss: 1.0240 - accuracy: 0.4848
Epoch 4/40
33/33 [==============================] - 0s 2ms/step - loss: 1.0227 - accuracy: 0.5000
Epoch 5/40
33/33 [==============================] - 0s 2ms/step - loss: 0.7902 - accuracy: 0.5182
Epoch 6/40
33/33 [==============================] - 0s 2ms/step - loss: 0.7821 - accuracy: 0.5212
Epoch 7/40
33/33 [==============================] - 0s 2ms/step - loss: 0.7482 - accuracy: 0.5121
Epoch 8/40
33/33 [==============================] - 0s 2ms/step - loss: 0.8121 - accuracy: 0.5030
Epoch 9/40
33/33 [==============================] - 0s 2ms/step - loss: 0.7778 - accuracy: 0.5152
Epoch 10/40
33/33 [==============================] - 0s 2ms/step - loss: 0.7022 - accuracy: 0.5667
Epoch 11/40
33/33 [

#Part9: Multi-Label classification with non-balanced data using Neural Network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense,Input,Dropout,Flatten,Conv2D,MaxPool2D

data_temp_2 = data3.copy()

y = pd.DataFrame([data_temp_2.pop(x) for x in ['Insominia', 'shizopherania', 'vascula_demetia', 'MBD', 'Bipolar']]).T

X = data_temp_2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_test, X_validation, y_train_test, y_validation = train_test_split(X_train, y_train, test_size=0.3)


model = Sequential()
model.add(Dense(15, input_dim=22, activation='relu', name='Input'))
model.add(Dropout(0.03))
model.add(Dense(20, activation='relu', name='Hidden1'))
model.add(Dropout(0.03))
model.add(Dense(20, activation='relu', name='Hidden2'))
model.add(Dropout(0.03))
model.add(Dense(40, activation='relu', name='Hidden3'))
model.add(Dropout(0.03))
model.add(Dense(5, activation='sigmoid', name='Output'))

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

X_train_np = np.asarray(X_train_test).astype('int')

X_train_tensor = tf.convert_to_tensor(X_train_np)
y_train_tensor = tf.convert_to_tensor(y_train_test)

model.fit(X_train_tensor, y_train_tensor, epochs=40, batch_size=10)


# evaluate the model with the validation data
X_validate_np = np.asarray(X_validation).astype('int')

X_validate_tensor = tf.convert_to_tensor(X_validate_np)
y_validate_tensor = tf.convert_to_tensor(y_validation)

print(f"Validation Loss and Accuracy is {model.evaluate(X_validate_tensor, y_validate_tensor, batch_size=10)}")

#evaluate the model with test data
X_test_np = np.asarray(X_test).astype('int')

X_test_tensor = tf.convert_to_tensor(X_test_np)
y_test_tensor = tf.convert_to_tensor(y_test)

print(f"Test Loss and Accuracy is {model.evaluate(X_test_tensor, y_test_tensor, batch_size=10)}")

Epoch 1/40
27/27 [==============================] - 1s 3ms/step - loss: 89.7104 - accuracy: 0.5593
Epoch 2/40
27/27 [==============================] - 0s 5ms/step - loss: 27.7085 - accuracy: 0.3852
Epoch 3/40
27/27 [==============================] - 0s 5ms/step - loss: 18.2493 - accuracy: 0.1704
Epoch 4/40
27/27 [==============================] - 0s 4ms/step - loss: 12.1192 - accuracy: 0.1926
Epoch 5/40
27/27 [==============================] - 0s 4ms/step - loss: 10.7160 - accuracy: 0.3259
Epoch 6/40
27/27 [==============================] - 0s 4ms/step - loss: 8.3538 - accuracy: 0.3037
Epoch 7/40
27/27 [==============================] - 0s 4ms/step - loss: 6.7871 - accuracy: 0.2667
Epoch 8/40
27/27 [==============================] - 0s 4ms/step - loss: 6.8634 - accuracy: 0.1704
Epoch 9/40
27/27 [==============================] - 0s 4ms/step - loss: 7.5292 - accuracy: 0.3074
Epoch 10/40
27/27 [==============================] - 0s 9ms/step - loss: 6.0548 - accuracy: 0.3519
Epoch 11/40
27

#Part10: Multi-Lablel classificaiton with balanced data using Neural Network

In [ ]:
from keras.utils import np_utils

data_temp = data2.copy()

y = data_temp.pop('target')

X = data_temp

# SMOTE the training data
smote = SMOTE(random_state=101)
X_new, y_new = smote.fit_resample(X, y.ravel())

y_new2 = []
for val in y_new.ravel():
  item = []
  for char in val:
    item.append(char)
  y_new2.append(item)

y_new3 = pd.DataFrame(y_new2, columns = ['Insominia', 'shizopherania', 'vascula_demetia', 'MBD', 'Bipolar']).astype('int')
  

X_train, X_test, y_train, y_test = train_test_split(X_new, y_new3, test_size=0.2)

X_train_test, X_validation, y_train_test, y_validation = train_test_split(X_train, y_train, test_size=0.3)



model = Sequential()
model.add(Dense(15, input_dim=21, activation='relu', name='Input'))
model.add(Dropout(0.1))
model.add(Dense(20, activation='relu', name='Hidden1'))
model.add(Dropout(0.1))
model.add(Dense(20, activation='relu', name='Hidden2'))
model.add(Dropout(0.1))
model.add(Dense(40, activation='relu', name='Hidden3'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='sigmoid', name='Output'))

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

X_train_np = np.asarray(X_train_test).astype('int')

X_train_tensor = tf.convert_to_tensor(X_train_np)
y_train_tensor = tf.convert_to_tensor(y_train_test)

model.fit(X_train_tensor, y_train_tensor, epochs=40, batch_size=10)


# evaluate the model with the validation data
X_validate_np = np.asarray(X_validation).astype('int')

X_validate_tensor = tf.convert_to_tensor(X_validate_np)
y_validate_tensor = tf.convert_to_tensor(y_validation)

print(f'Validation Loss and Accuracy is {model.evaluate(X_validate_tensor, y_validate_tensor, batch_size=10)}')


X_test_np = np.asarray(X_test).astype('int')

X_test_tensor = tf.convert_to_tensor(X_test_np)
y_test_tensor = tf.convert_to_tensor(y_test)

print(f'Test Loss and Accuracy is {model.evaluate(X_test_tensor, y_test_tensor, batch_size=10)}')

Epoch 1/40
68/68 [==============================] - 1s 2ms/step - loss: 0.8051 - accuracy: 0.2301
Epoch 2/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.2596
Epoch 3/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6811 - accuracy: 0.2714
Epoch 4/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6629 - accuracy: 0.3260
Epoch 5/40
68/68 [==============================] - 0s 3ms/step - loss: 0.6655 - accuracy: 0.3673
Epoch 6/40
68/68 [==============================] - 0s 3ms/step - loss: 0.6560 - accuracy: 0.3909
Epoch 7/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6418 - accuracy: 0.3805
Epoch 8/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6404 - accuracy: 0.3732
Epoch 9/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6222 - accuracy: 0.4366
Epoch 10/40
68/68 [==============================] - 0s 2ms/step - loss: 0.6188 - accuracy: 0.4189
Epoch 11/40
68/68 [